# PL12 .- HYBRID QUANTUM NEURAL NETWORKS

En este notebook, vamos a mostrar cómo se pueden construir redes neuronales cuánticas híbridas integrando *pennylane* con *Keras*.

Comenzamos importando las librerías necesarias y fijando las semillas de los generadores aleatorios.

In [ ]:
# !pip install pennylane
# !pip install tensorflow==2.15
# !pip install silence_tensorflow

import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import pennylane as qml
import random
import os

os.environ['PYTHONHASHSEED']='0'

rs = 12345  # Semilla aleatoria

random.seed(rs)
np.random.seed(rs)
tf.random.set_seed(rs)

from silence_tensorflow import silence_tensorflow
silence_tensorflow()

Creamos un conjunto de datos sencillo y los dividimos en entrenamiento y test.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=200, centers = [[0,2],[2,0]], random_state = rs)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=rs)
plt.scatter(X[:, 0], X[:, 1], c = y, cmap=matplotlib.colors.ListedColormap(["red","green"]));

Como vamos a usar *angle embedding*, escalamos los datos al intervalo $[0,\pi]$.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,np.pi))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## REDES NEURONALES HÍBRIDAS

Cuando tenemos una capa de Keras que es, en realidad, un circuito cuántico, se abre la posibilidad de combinar capas clásicas y cuánticas: es lo que llamamos redes neuronales híbridas.

Para crearlas, podemos simplemente usar el modelo secuencial de Keras y combinar capas de un tipo y de otro, teniendo en cuenta el número de entradas y salidas que tiene cada una. Una ventaja de este enfoque es que podemos fácilmente reducir el número de atributos con una primera capa clásica.

Nótese que en el circuito cuántico ahora devolvemos tantos valores como qubits hay.

Empezamos con una sola repetición de la forma variacional en la capa cuántica.

In [ ]:
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

n_layers = 1
weight_shapes = {"weights": (n_layers, n_qubits, 3)}

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(n_qubits,activation='relu',input_dim=2))
model.add(qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

opt = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(loss='binary_crossentropy', optimizer = opt, metrics=["accuracy"])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_split=0.2, callbacks=[early_stopping_cb])

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(8, 5));

model.evaluate(X_test, y_test)

In [ ]:
model.summary()

Ahora, usamos dos repeticiones de la forma variacional.

In [ ]:
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

n_layers = 2
weight_shapes = {"weights": (n_layers, n_qubits, 3)}

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(n_qubits,activation='relu',input_dim=2))
model.add(qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

opt = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(loss='binary_crossentropy', optimizer = opt, metrics=["accuracy"])

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100,
                    validation_split=0.2, callbacks=[early_stopping_cb])

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5));

model.evaluate(X_test, y_test)

In [ ]:
model.summary()

## EJERCICIO

Crear una red híbrida para un problema de clasificación binaria con 50 variables de entrada (por ejemplo, con *make_classification*, de *SciKit Learn*). Usar una capa cuántica de 8 qubits.